## Preperations
Loading libraries and the data

In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.decomposition import PCA
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
import datetime

import os, math

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Preprocessing
from sklearn.preprocessing import MinMaxScaler
# Algorithms
from minisom import MiniSom
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans

from sklearn.decomposition import PCA

In [ ]:
with open("IS-thesis-pattern-extraction/options.txt", 'r') as f:
    options = f.readlines()
    options = {option.split("=")[0]: option.split("=")[1].strip() for option in options}
print(options)

In [ ]:
# From JSON file
# RUG = pd.read_json(f'{options["RUG_dir"]}/summary.json')
# display(RUG)
# RUG.to_pickle(f'{options["RUG_dir"]}/summary.pkl')

# From pickl file


RUG = pd.read_csv("rug_csv.csv", nrows=100000)

column_list = [...]
RUG.columns = column_list
RUG
# RUG.columns = [i.replace(r'\\PIAF\Module Database\Vitens', "") for i in RUG.columns]

Problem: if the data ends in a different daylight savings time phase, the function gets confused and errors

In [ ]:
RUG.date = pd.date_range(start = RUG.date.min(), end = RUG.date.max(), freq='min')

RUG.set_index('date', inplace=True)

In [ ]:
display(RUG.apply(lambda x: x.isnull().sum()))

In [ ]:

# split data into groups of timeperiod x (here 1d)
groups = RUG['...'].groupby(pd.Grouper(freq='D'))

# get the calender date of the groups
days = list(groups.first().index.strftime('%Y:%m:%d'))

gro = [groups.get_group(x).reset_index(drop=True) for x in groups.groups]

temp = pd.concat(gro, axis=1, keys=days)

temp.index = pd.date_range("00:00", "23:59", freq="1min").strftime('%H:%M')
temp


In [ ]:
temp = temp[::2]

## Scaling

In [ ]:
scaled_list = [temp[col] for col in temp]

In [ ]:
for i in range(len(scaled_list)):
    scaler = MinMaxScaler()
    scaled_list[i] = MinMaxScaler().fit_transform(scaled_list[i].to_numpy().reshape(-1, 1))
    scaled_list[i]= scaled_list[i].reshape(len(scaled_list[i]))

scaled_list = np.nan_to_num(scaled_list, copy=False, nan=0.0)
scaled_list

## Clustering

In [ ]:
som_x = som_y = math.ceil(math.sqrt(math.sqrt(len(scaled_list))))
# I didn't see its significance but to make the map square,
# I calculated square root of map size which is 
# the square root of the number of series
# for the row and column counts of som

som = MiniSom(som_x, som_y,len(scaled_list[0]), sigma=0.3, learning_rate = 0.1)

som.random_weights_init(scaled_list)
som.train(scaled_list, 50000)

In [ ]:
# Little handy function to plot series
def plot_som_series_averaged_center(som_x, som_y, win_map):
    fig, axs = plt.subplots(som_x,som_y,figsize=(25,25))
    fig.suptitle('Clusters')
    for x in range(som_x):
        for y in range(som_y):
            cluster = (x,y)
            if cluster in win_map.keys():
                for series in win_map[cluster]:
                    axs[cluster].plot(series,c="gray",alpha=0.5) 
                axs[cluster].plot(np.average(np.vstack(win_map[cluster]),axis=0),c="red")
            cluster_number = x*som_y+y+1
            axs[cluster].set_title(f"Cluster {cluster_number}")

    plt.show()

In [ ]:
win_map = som.win_map(scaled_list)
# Returns the mapping of the winner nodes and inputs

plot_som_series_averaged_center(som_x, som_y, win_map)

In [ ]:

cluster_c = []
cluster_n = []
for x in range(som_x):
    for y in range(som_y):
        cluster = (x,y)
        if cluster in win_map.keys():
            cluster_c.append(len(win_map[cluster]))
        else:
            cluster_c.append(0)
        cluster_number = x*som_y+y+1
        cluster_n.append(f"Cluster {cluster_number}")

plt.figure(figsize=(25,5))
plt.title("Cluster Distribution for SOM")
plt.bar(cluster_n,cluster_c)
plt.show()

In [ ]:
cluster_count = math.ceil(math.sqrt(len(scaled_list))) 
# A good rule of thumb is choosing k as the square root of the number of points in the training data set in kNN

km = TimeSeriesKMeans(n_clusters=cluster_count, metric="dtw")

labels = km.fit_predict(scaled_list)

In [ ]:
plot_count = math.ceil(math.sqrt(cluster_count))

fig, axs = plt.subplots(plot_count,plot_count,figsize=(25,25))
fig.suptitle('Clusters')
row_i=0
column_j=0
# For each label there is,
# plots every series with that label
for label in set(labels):
    cluster = []
    for i in range(len(labels)):
            if(labels[i]==label):
                axs[row_i, column_j].plot(scaled_list[i],c="gray",alpha=0.4)
                cluster.append(scaled_list[i])
    if len(cluster) > 0:
        axs[row_i, column_j].plot(np.average(np.vstack(cluster),axis=0),c="red")
    axs[row_i, column_j].set_title("Cluster "+str(row_i*som_y+column_j))
    column_j+=1
    if column_j%plot_count == 0:
        row_i+=1
        column_j=0
        
plt.show()

## Test